# Introduction
Deep neural models based on Convolutional Neural Networks (CNNs) have enabled unprecedented breakthroughs
in a variety of computer vision tasks, image classification, object detection, image segmentation, image captioning and more. While these models enable superior performance, they considered to be a black boxes and hard to interpret.

**Why interpretability matters?**
>In order to build trust in intelligent
systems and move towards their meaningful integration into our everyday lives, it is clear that we must build ‘transparent’ models that have the ability to explain why they predict what they predict.


## APPROACH
Gradient-weighted Class Activation Mapping
(Grad-CAM), uses the gradients of any target concept (say
‘dog’ in a classification network or a sequence of words
in captioning network) flowing into the final convolutional
layer to produce a coarse localization map highlighting the
important regions in the image for predicting the concept.

To obtain the class discriminative localization map of width u and height v for any class c, we first compute the gradient of the score for the class c, yc (before the softmax) with respect to feature maps Ak of a convolutional layer.

![alt text](https://raw.githubusercontent.com/omarsayed7/Grad-CAM/master/imgs/fig1.JPG)

After calculating ak for the target class c, we perform a weighted combination of activation maps and follow it by ReLU.
![alt text](https://raw.githubusercontent.com/omarsayed7/Grad-CAM/master/imgs/fig2.JPG)

We will demonstrate this technique using the pre-trained resnet18 network in both Keras and PyTorch. 

# PyTorch Implementation

In [0]:
from torchvision import datasets, models, transforms
from torchsummary import summary 

In [7]:
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [8]:
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [0]:
def grad_cam_torch():
  pass
  

# Keras Implementation 

In [12]:
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions
import keras.backend as K

K.clear_session()

model = ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 3s 0us/step


In [13]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
def grad_cam_keras():
  pass
  